In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import StandardScaler

In [2]:
import tensorflow as tf

In [3]:
df= pd.read_csv('/dbfs/FileStore/tables/numericTargetMail.csv') #change filename according to your actual data file
df.head()

In [4]:
def processInputData(df, testing_size = 1000):
  features = []
  row, col = df.shape
  for r in range(row):
    feature = []
    for c in range(col-1):
      feature.append(df.iloc[r,c])
      if df.iloc[r,col-1] == 1:
        features.append([feature,[1,0]])
      else:
        features.append([feature,[0,1]])
  random.shuffle(features)
  features = np.array(features)
  train_x = list(features[:,0][:len(df)-testing_size])
  train_y = list(features[:,1][:len(df)-testing_size])
  test_x = list(features[:,0][-testing_size:])
  test_y = list(features[:,1][-testing_size:])
  return train_x, train_y, test_x, test_y

train_x,train_y,test_x,test_y = processInputData(df)

In [5]:
sc = StandardScaler()
sc.fit(train_x)
train_x_std = sc.transform(train_x)
test_x_std = sc.transform(test_x)

In [6]:
n_nodes_input_layer = 11 # Number of neurons at input layer
n_nodes_hl1 = 500        # Number of neurons at hidden layers
n_nodes_output_layer = 2 # Number of neurons at output layer

batch_size = 100

training_iterations = 20

x = tf.placeholder('float')
y = tf.placeholder('float')

hidden_1_layer = {'f_fum':n_nodes_hl1,
                  'weight':tf.Variable(tf.random_normal([n_nodes_input_layer, n_nodes_hl1])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl1]))}

output_layer = {'f_fum':None,
                'weight':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_output_layer])),
                'bias':tf.Variable(tf.random_normal([n_nodes_output_layer])),}


#define neural network with 1 hidden layer
def neural_network_model(data):

    l1 = tf.add(tf.matmul(data,hidden_1_layer['weight']), hidden_1_layer['bias'])
    l1 = tf.nn.relu(l1)

    output = tf.matmul(l1,output_layer['weight']) + output_layer['bias']

    return output
  


In [7]:
def train_NN(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
    conf_matrix = [[0,0],[0,0]]
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for epoch in range(training_iterations):
            epoch_loss = 0
            i=0
            while i < len(train_x_std):
                start = i
                end = i+batch_size
                batch_x = np.array(train_x_std[start:end])
                batch_y = np.array(train_y[start:end])

                _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                              y: batch_y})
                epoch_loss += c
                i+=batch_size

            print('Iteration', epoch+1, 'completed out of',training_iterations,'loss function value:',epoch_loss)
        correct = tf.equal(tf.argmax(prediction, 1),tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct,'float'))
        yp = tf.cast(tf.argmax(prediction, 1),'float')
        print('Accuracy:',accuracy.eval({x:test_x_std, y:test_y}))
        ypred = yp.eval({x:test_x_std, y:test_y})
        cf = [[0,0],[0,0]]
        tp = 0
        tn = 0
        fp = 0
        fn = 0
        for ind in range(len(ypred)):
            if (ypred[ind] == 1) and (test_y[ind][0] == 1):
                tp += 1
            if (ypred[ind] == 1) and (test_y[ind][0] == 0):
                fp += 1
            if (ypred[ind] == 0) and (test_y[ind][0] == 1):
               fn += 1
            if (ypred[ind] == 0) and (test_y[ind][0] == 0):
               tn += 1
        conf_matrix[0][0] = tp
        conf_matrix[0][1] = fp
        conf_matrix[1][0] = fn
        conf_matrix[1][1] = tn
        print('confusion matrix:',conf_matrix)

In [8]:
train_NN(x)

In [9]:
#define neural network structure
n_nodes_input_layer = 11
n_nodes_hl1 = 250
n_nodes_hl2 = 250
n_nodes_output_layer = 2

batch_size = 100

training_iterations = 20

x = tf.placeholder('float')
y = tf.placeholder('float')
hidden_1_layer = {'f_fum':n_nodes_hl1,
                  'weight':tf.Variable(tf.random_normal([n_nodes_input_layer, n_nodes_hl1])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl1]))}
hidden_2_layer = {'f_fum':n_nodes_hl2,
                  'weight':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl2]))}
output_layer = {'f_fum':None,
                'weight':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_output_layer])),
                'bias':tf.Variable(tf.random_normal([n_nodes_output_layer])),}
#define neural network model using tensorflow
def neural_network_model(data):
    l1 = tf.add(tf.matmul(data,hidden_1_layer['weight']), hidden_1_layer['bias'])
    l1 = tf.nn.relu(l1)
    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weight']), hidden_2_layer['bias'])
    l2 = tf.nn.relu(l2)

    output = tf.matmul(l2,output_layer['weight']) + output_layer['bias']
    return output

In [10]:
train_NN(x)